In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

from utils import format_node_names, get_list

#jupyter-notebook --no-browser --ip=0.0.0.0 --port 8888

In [2]:
"""
First run preprocessing notebook in this direcoty.
This notebook clean the data that we have obtained from Slurm. 
    """

'\nFirst run preprocessing notebook in this direcoty.\nThis notebook clean the data that we have obtained from Slurm. \n    '

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
folder_path = Path('./system_analytics_2024/slurm_data')
file_path_parquet_reading_slurm_data= folder_path / 'slurm_data_preprocessed.parquet.gzip'
prom_node_list = Path('./system_analytics_2024/prom_data') / 'prom_instance_list.txt'

In [4]:
df = pd.read_parquet(file_path_parquet_reading_slurm_data)
# df.rename(columns={"JobID":"Slurm_job_id"}, inplace=True)
df.sample(n=10)


,job_id,Submit,Eligible,Start,End,Elapsed,Slurm_job_id,State,AllocCPUS,TotalCPU,AveRSS,MaxRSS,NodeList
2,8020315,2024-10-01T12:11:27,2024-10-01T12:11:27,2024-10-01T12:11:27,2024-10-01T12:30:58,00:19:31,8020315.ext+,COMPLETED,32,00:00.001,188K,188K,tcn142
1,8251679,2024-10-23T15:44:00,2024-10-23T15:44:00,2024-10-23T15:44:00,2024-10-23T15:44:18,00:00:18,8251652_22.+,FAILED,16,00:03.315,1571K,1571K,tcn178
20,8073607,2024-10-04T13:46:01,2024-10-04T13:46:01,2024-10-04T13:46:01,2024-10-05T13:27:51,23:41:50,8073607.17,COMPLETED,1,23:28:09,813585K,813585K,fcn40
146,8155778,2024-10-11T19:02:13,2024-10-11T19:02:13,2024-10-11T19:02:13,2024-10-11T19:02:28,00:00:15,8155778_49.+,COMPLETED,16,00:00.001,141K,141K,tcn93
0,4429565,2023-11-13T03:18:51,2023-11-13T03:18:51,2023-11-13T03:19:00,2023-11-13T03:19:18,00:00:18,4429565,COMPLETED,384,00:00:00,None,None,"tcn[671,675]"
0,6010503,2024-04-22T09:26:12,2024-04-22T09:26:18,2024-04-22T15:08:25,2024-04-22T15:12:51,00:04:26,5997551_8383,COMPLETED,32,00:00:00,None,None,fcn41
1,8004806,2024-09-30T11:09:39,2024-09-30T11:09:39,2024-09-30T11:09:39,2024-09-30T11:10:02,00:00:23,8004806.bat+,COMPLETED,16,00:06.877,1538K,1538K,tcn124
3,4289203,2023-10-30T20:30:23,2023-10-30T20:30:24,2023-10-30T20:35:14,2023-10-30T21:02:29,00:27:15,4289203_4,COMPLETED,128,00:00:00,None,None,tcn49
0,7027666,2024-07-16T16:24:10,2024-07-16T16:24:10,2024-07-16T20:13:51,2024-07-16T20:14:22,00:00:31,7027666,COMPLETED,16,00:00:00,None,None,tcn219
505,6176939,2024-05-07T17:00:08,2024-05-07T17:00:10,2024-05-07T17:35:07,2024-05-07T17:46:05,00:10:58,6176939_506,COMPLETED,16,00:00:00,None,None,tcn390


In [5]:
""" 
ISSUE: 
We have a probelm with how pands see the rows in the data frame.
Here we are checking for the number for duplicated rows and get a sample fo them.
While pandas says we have over 1000 of duplicated rows but in the sample we can see that they are not
duplicated at all. ==>needs to be checked
"""

print(f"How many rows are duplicated in pandas view: {df.duplicated().sum()}")
df[df.duplicated()].head()

How many rows are duplicated in pandas view: 1036


,job_id,Submit,Eligible,Start,End,Elapsed,Slurm_job_id,State,AllocCPUS,TotalCPU,AveRSS,MaxRSS,NodeList
2981,7715376,2024-09-03T17:04:29,2024-09-03T17:04:29,2024-09-03T17:04:29,2024-09-03T17:08:21,00:03:52,7715376_799+,COMPLETED,32,00:00.001,146K,146K,fcn62
1034,7720963,2024-09-04T09:55:14,2024-09-04T09:55:14,2024-09-04T09:55:14,2024-09-04T09:57:18,00:02:04,7720963_934+,COMPLETED,32,00:00.001,149K,149K,fcn62
1376,7720963,2024-09-04T09:57:41,2024-09-04T09:57:41,2024-09-04T09:57:41,2024-09-04T10:00:48,00:03:07,7720963_945+,COMPLETED,32,00:00.001,147K,147K,fcn63
1370,7728324,2024-09-04T13:54:18,2024-09-04T13:54:18,2024-09-04T13:54:18,2024-09-04T13:55:52,00:01:34,7728324_134+,COMPLETED,32,00:00.001,149K,149K,fcn40
194,7744649,2024-09-06T13:36:51,2024-09-06T13:36:51,2024-09-06T13:36:51,2024-09-06T13:36:52,00:00:01,7744649_17.+,COMPLETED,1,00:00.359,1535K,1535K,tcn192


In [6]:
"""
ISSUE: 
    Again we have a probelm here. We are aksing for how many duplicated rows
    we have based on the JobID that slurm returns. It seems that we have more than a million, 
    but it seems that they are not at all
    """

print(f"How many JobIDs are duplicated in pandas view: {df['Slurm_job_id'].duplicated().sum()}")
df[df.duplicated('Slurm_job_id')].head()

How many JobIDs are duplicated in pandas view: 1422706


,job_id,Submit,Eligible,Start,End,Elapsed,Slurm_job_id,State,AllocCPUS,TotalCPU,AveRSS,MaxRSS,NodeList
14,7617959,2024-08-30T12:46:11,2024-08-30T12:46:11,2024-08-30T12:46:11,2024-09-01T01:12:34,1-12:26:23,7617959_13.+,COMPLETED,128,00:00.001,193K,193K,tcn335
26,7617959,2024-08-31T03:17:30,2024-08-31T03:17:30,2024-08-31T03:17:30,2024-09-01T09:29:34,1-06:12:04,7617959_23.+,COMPLETED,128,00:00.001,186K,186K,tcn20
29,7617959,2024-08-31T06:29:24,2024-08-31T06:29:24,2024-08-31T06:29:24,2024-09-01T02:12:39,19:43:15,7617959_24.+,COMPLETED,128,00:00.001,181K,181K,tcn4
32,7617959,2024-08-31T11:53:41,2024-08-31T11:53:41,2024-08-31T11:53:41,2024-09-01T04:05:20,16:11:39,7617959_25.+,COMPLETED,128,00:00.001,181K,181K,tcn83
35,7617959,2024-08-31T22:34:26,2024-08-31T22:34:26,2024-08-31T22:34:26,2024-09-01T02:04:21,03:29:55,7617959_26.+,COMPLETED,128,00:00.001,306K,306K,tcn237


In [7]:
"""
It would be great to remove all the data node list that end in + sign
since they are not clear to me which nodes they are refering to.
"""

df['NodeList_match'] = df['NodeList'].str.endswith("+")
print(f"How many strange nodes: {df['NodeList_match'].sum()}")
# removing them
df = df[~df['NodeList_match']]
# note that we have some special format for the nodes:
display(df['NodeList'].value_counts()[-10:-1])
# we have to process them further. 
df['formatted_node_names'] = df['NodeList'].apply(format_node_names)
display(df['formatted_node_names'].value_counts()[-10:-1])


How many strange nodes: 48318


NodeList
tcn[8,17]         1
gcn[8,71-72]      1
tcn[909,919]      1
tcn[106-113]      1
tcn[48,61]        1
tcn[694,734]      1
tcn[1098-1099]    1
tcn[931-934]      1
tcn[920,930]      1
Name: count, dtype: int64

formatted_node_names
tcn8,tcn17                                                 1
gcn8,gcn71,gcn72                                           1
tcn909,tcn919                                              1
tcn106,tcn107,tcn108,tcn109,tcn110,tcn111,tcn112,tcn113    1
tcn48,tcn61                                                1
tcn694,tcn734                                              1
tcn1098,tcn1099                                            1
tcn931,tcn932,tcn933,tcn934                                1
tcn920,tcn930                                              1
Name: count, dtype: int64

In [8]:
""" 
ASSUMPTION: We assume that when a record is like this:


jobid  start_time  end_time    node_names
123     1           100          [fcn10, fcn11]


the it can be rewrriten like this

jobid  start_time  end_time    node_names
123     1           100          fcn10
123     1           100          fcn11



This allows us to have a better understanding of the node specific features. 
"""



len_list = [len(name.split(',')) for name in df['formatted_node_names']]
df['Number_of_nodes'] = len_list
node_names = [name.split(',') for name in df['formatted_node_names']]
nodes = [node for job_node in node_names for node in job_node]
ser_list = [df.iloc[:,i].repeat(len_list) for i in range(len(df.columns))]
del df
df = pd.concat(ser_list, axis=1)
df['node_id'] = nodes
df[['NodeList', 'formatted_node_names', 'node_id']].sample(n=10)


,NodeList,formatted_node_names,node_id
2,tcn779,tcn779,tcn779
31,gcn25,gcn25,gcn25
0,tcn95,tcn95,tcn95
0,tcn441,tcn441,tcn441
15,tcn176,tcn176,tcn176
0,tcn664,tcn664,tcn664
0,tcn669,tcn669,tcn669
0,tcn480,tcn480,tcn480
0,tcn1121,tcn1121,tcn1121
2,tcn161,tcn161,tcn161


In [9]:
"""
    EXCEPT for 'gcn1', all the Promethues nodes are in the SLURM data that we have!
   But, it is iteresting that around 50 nodes are not reporting any 
    measurement in Prom. database! These are mainly tcn1270-tcn1320
    """
# prom nodes, we obtained by querying Prom.
prom_nodes = get_list(prom_node_list)
slurm_nodes = df['node_id'].unique()

slurm_exclusive_nodes = list(set(slurm_nodes) - set(prom_nodes))
prom_exclusive_nodes = list(set(prom_nodes)- set(slurm_nodes))

print(f"sample of SLURM exclusice nodes: {slurm_exclusive_nodes[0:5]}", '\n',
      f"length of SLURM exclusive nodes: {len(slurm_exclusive_nodes)}")

print(f"sample of prom exclusive nodes: {prom_exclusive_nodes[0:5]}", '\n',
      f"lenght of prom exclusive nodes: {len(prom_exclusive_nodes)}")


sample of SLURM exclusice nodes: ['tcn1299', 'tcn1298', 'tcn1264', 'tcn1279', 'tcn1281'] 
 length of SLURM exclusive nodes: 50
sample of prom exclusive nodes: ['gcn1'] 
 lenght of prom exclusive nodes: 1


In [10]:
"""
Creating a node tag based on the name of the nodes.
This is beacuse e.g, gcn nodes usually run ML jobs.
    """
# put the node type based on the node names
df['node_type'] = df['node_id'].str[0:3]

df_node_type_count = pd.DataFrame(df.groupby('node_type')['node_id'].nunique()).reset_index()
df_node_type_count.columns =['node_type', 'number_of_nodes']
display(df_node_type_count.head(n=10))
print(f"total number of nodes:{(df_node_type_count.sum()['number_of_nodes'])}")

,node_type,number_of_nodes
0,fcn,120
1,gcn,159
2,hcn,4
3,srv,2
4,tcn,1311


total number of nodes:1596


In [11]:

""" 
Here we check the format for the date columns. Some of the values are not
in the expected format. We focus on the End_match signal on remove those that
does not have the format.
"""
pattern = r"\d{4}-\d{2}-\d{2}[T]\d{2}:\d{2}:\d{2}"
# Check if each timestamp matches the pattern
df['Start_match'] = df['Start'].str.match(pattern)
df['End_match'] = df['End'].str.match(pattern)
df['Submit_match'] = df['Submit'].str.match(pattern)
df['Eligible_match'] = df['Eligible'].str.match(pattern)

print(df['Start_match'].sum(), df['End_match'].sum(),
      df['Submit_match'].sum(), df['Eligible_match'].sum(), len(df))


df = df[df['End_match']]

5748593 5748474 5748593 5748478 5748593


In [12]:
# turn the data column to pandas date column

"""
   Add features that you like here and then use them in the next block! 
    """
df['Submit'] = pd.to_datetime(df['Submit'])
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df['job_run_time'] = df['End'] - df['Start']
df['job_waiting_time'] = df['Start'] - df['Submit']




In [13]:
# drop the srv nodes!
df = df[~(df['node_type']=='srv')]

# drop some singals
df.drop(['NodeList_match', 'formatted_node_names', 'Start_match',
         'End_match', 'NodeList', 'Submit_match','Eligible_match'], axis=1, inplace=True)

# save the file
#df.to_parquet(folder_path / 'slurm_data_cleaned.parquet.gzip', compression='gzip')

In [14]:

""" 
We analyse the status of a job based on its duration.
The interesting obeservation here is that for as the job duration
takes a longer time the channce of not completed increases. 
This has been obsereved before as well.

"""



# time_for_cutting = [pd.Timedelta('5min'), 
#                     pd.Timedelta('1h'), pd.Timedelta('6h'),
#                      pd.Timedelta('1day'), pd.Timedelta('500day')]
# labels = pd.cut(df['job_run_time'], bins=time_for_cutting, right=False)
# df['cat'] = labels

# # this is exactly the behavior that we want to show.
# df_temp = pd.DataFrame((df.groupby(['cat'])['State'].value_counts(normalize=True)))
# df_temp.reset_index(inplace=True)


# df_stacked = df_temp.pivot_table(index='cat', columns='State')
# df_stacked = df_stacked.droplevel(level=0, axis=1).copy()
# df_stacked.index.name = ''
# df_stacked.index = ['<1hour', '<6hour', '<1 day', '>1day']
# plt.figure(figsize=(20, 6))
# df_stacked.plot(kind='bar', stacked=True, alpha=0.9)


# # sns.barplot(df_temp, x='cat', y='proportion', hue='State' )



' \nWe analyse the status of a job based on its duration.\nThe interesting obeservation here is that for as the job duration\ntakes a longer time the channce of not completed increases. \nThis has been obsereved before as well.\n\n'